In [1]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv('C:/Users/Enzo/OneDrive/Escritorio/Data Science Henry/Modulo 3/Avance 1/VentasTransformed.csv')

In [19]:
df

,VentaID,Año,Trimestre,Mes,Día,HoraVenta,NombreSucursal,Ciudad,VendedorNombre,ClienteNombre,...,CantidadProducto2,PrecioUnitarioProducto2,SubtotalProducto2,NombreProducto3,MarcaProducto3,CantidadProducto3,PrecioUnitarioProducto3,SubtotalProducto3,DescuentoVenta,TotalVenta
0,1,2015,Qtr 4,diciembre,31,1899-12-30 05:42:43,Pereira,Pereira,Amílcar Ortega-Alberto,Bienvenida Nebot-Fiol,...,2,8000000,16000000,Dell Latitude 7420,Dell,1,5600000,5600000,0.00,31200000
1,2,2019,Qtr 1,marzo,23,1899-12-30 19:03:21,Medellín 1,Medellín,Ana Sofía Llopis Blázquez,Teófila Bueno-Novoa,...,1,6800000,6800000,NaN,NaN,0,0,0,0.00,10800000
2,3,2018,Qtr 4,diciembre,23,1899-12-30 02:32:34,Medellín 2,Medellín,Juan José Porcel-Riera,Gilberto Chamorro Catalá,...,1,7200000,7200000,HP Pavilion 15,HP,1,3500000,3500000,0.00,21900000
3,4,2015,Qtr 4,noviembre,14,1899-12-30 00:37:36,Medellín 2,Medellín,Juan José Porcel-Riera,Máximo Coronado Huerta,...,2,4400000,8800000,NaN,NaN,0,0,0,0.00,13600000
4,5,2016,Qtr 4,noviembre,29,1899-12-30 10:34:20,Cali,Cali,Jacinta Juárez Marín,Yago Oliver,...,1,3500000,3500000,Microsoft Surface Laptop 4,Microsoft,1,5200000,5200000,0.00,13900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,2020,Qtr 4,diciembre,31,1899-12-30 01:33:17,Medellín 2,Medellín,Juan José Porcel-Riera,Claudio del Amo,...,0,0,0,NaN,NaN,0,0,0,0.00,6800000
29996,29997,2017,Qtr 3,julio,14,1899-12-30 11:29:20,Cali,Cali,Basilio Marcos Barrena,Aurora Alegria Castellanos,...,2,6800000,13600000,Asus TUF Gaming A15,Asus,1,4800000,4800000,0.00,26400000
29997,29998,2019,Qtr 2,abril,6,1899-12-30 21:38:27,Medellín 2,Medellín,Claudia Sastre-Quintero,Paco Olivera Suárez,...,0,0,0,NaN,NaN,0,0,0,0.00,5600000
29998,29999,2024,Qtr 4,diciembre,27,1899-12-30 06:49:00,Medellín 1,Medellín,Dominga del Alcántara,Amílcar Amigó Barrio,...,1,4000000,4000000,NaN,NaN,0,0,0,0.00,10800000


In [24]:
# --- TABLA: CIUDADES ---
dim_ciudad = df[['Ciudad']].drop_duplicates().reset_index(drop=True)
dim_ciudad['CiudadID'] = dim_ciudad.index + 1

# --- TABLA: SUCURSALES ---
dim_sucursal = df[['NombreSucursal', 'Ciudad']].drop_duplicates().reset_index(drop=True)
dim_sucursal = dim_sucursal.merge(dim_ciudad, on='Ciudad', how='left')
dim_sucursal['SucursalID'] = dim_sucursal.index + 1

# --- TABLA: VENDEDORES ---
dim_vendedor = df[['VendedorNombre']].drop_duplicates().reset_index(drop=True)
dim_vendedor['VendedorID'] = dim_vendedor.index + 1

# --- TABLA: CLIENTES ---
dim_cliente = df[['ClienteNombre', 'GeneroCliente', 'EdadCliente', 'EmailCliente',
                  'TelefonoCliente', 'DireccionCliente']].drop_duplicates().reset_index(drop=True)
dim_cliente['ClienteID'] = dim_cliente.index + 1

# --- TABLA: MÉTODO DE PAGO ---
dim_metodo = df[['MetodoPago']].drop_duplicates().reset_index(drop=True)
dim_metodo['MetodoPagoID'] = dim_metodo.index + 1

In [25]:
# --- TABLA: PRODUCTOS ---
# Reorganizamos los productos (1, 2 y 3)
productos = []
for i in range(1, 4):
    temp = df[[f'NombreProducto{i}', f'MarcaProducto{i}', f'PrecioUnitarioProducto{i}']].drop_duplicates()
    temp.columns = ['NombreProducto', 'MarcaProducto', 'PrecioUnitario']
    productos.append(temp)

dim_producto = pd.concat(productos).drop_duplicates().reset_index(drop=True)
dim_producto['ProductoID'] = dim_producto.index + 1

In [28]:
# --- TABLA: FACTURAS ---
facturas_cols = [
    'VentaID', 'Año', 'Trimestre', 'Mes', 'Día', 'HoraVenta',
    'NombreSucursal', 'VendedorNombre', 'ClienteNombre', 'MetodoPago',
    'DescuentoVenta', 'TotalVenta'
]

facturas = df[facturas_cols].drop_duplicates().reset_index(drop=True)

# --- Vinculamos claves foráneas ---
facturas = (
    facturas
    .merge(dim_sucursal[['NombreSucursal', 'SucursalID']], on='NombreSucursal', how='left')
    .merge(dim_vendedor[['VendedorNombre', 'VendedorID']], on='VendedorNombre', how='left')
    .merge(dim_cliente[['ClienteNombre', 'ClienteID']], on='ClienteNombre', how='left')
    .merge(dim_metodo[['MetodoPago', 'MetodoPagoID']], on='MetodoPago', how='left')
)

# --- Eliminamos las columnas originales de texto ---
facturas = facturas.drop(columns=['NombreSucursal', 'VendedorNombre', 'ClienteNombre', 'MetodoPago'])

# --- Generamos la clave primaria ---
facturas['FacturaID'] = facturas.index + 1

# --- Reordenamos columnas ---
facturas = facturas[[
    'FacturaID', 'VentaID', 'Año', 'Trimestre', 'Mes', 'Día', 'HoraVenta',
    'SucursalID', 'VendedorID', 'ClienteID', 'MetodoPagoID',
    'DescuentoVenta', 'TotalVenta'
]]


In [29]:
# --- TABLA: DETALLE FACTURAS ---
detalles = []
for i in range(1, 4):
    temp = df[['VentaID', f'NombreProducto{i}', f'CantidadProducto{i}',
               f'PrecioUnitarioProducto{i}', f'SubtotalProducto{i}']].copy()
    temp.columns = ['VentaID', 'NombreProducto', 'Cantidad', 'PrecioUnitario', 'Subtotal']
    detalles.append(temp)

detalle_facturas = pd.concat(detalles).dropna(subset=['NombreProducto'])
detalle_facturas = detalle_facturas.merge(facturas[['VentaID', 'FacturaID']], on='VentaID', how='left')
detalle_facturas = detalle_facturas.merge(dim_producto[['NombreProducto', 'ProductoID']], on='NombreProducto', how='left')
detalle_facturas = detalle_facturas[['FacturaID', 'ProductoID', 'Cantidad', 'PrecioUnitario', 'Subtotal']]


In [30]:
# --- EXPORTAR TODO A UN SOLO ARCHIVO EXCEL ---
with pd.ExcelWriter('Modelo_Ventas.xlsx', engine='openpyxl') as writer:
    dim_producto.to_excel(writer, sheet_name='Productos', index=False)
    detalle_facturas.to_excel(writer, sheet_name='DetalleFacturas', index=False)
    dim_cliente.to_excel(writer, sheet_name='Clientes', index=False)
    dim_ciudad.to_excel(writer, sheet_name='Ciudades', index=False)
    dim_sucursal.to_excel(writer, sheet_name='Sucursales', index=False)
    facturas.to_excel(writer, sheet_name='Facturas', index=False)
    dim_vendedor.to_excel(writer, sheet_name='Vendedores', index=False)
    dim_metodo.to_excel(writer, sheet_name='MetodoPago', index=False)

print("✅ Archivo 'Modelo_Ventas.xlsx' generado con éxito.")


✅ Archivo 'Modelo_Ventas.xlsx' generado con éxito.


In [31]:
# --- VALIDAR INTEGRIDAD REFERENCIAL ---

print("\n🔍 Validando integridad referencial...\n")

# 1. Verificar que todos los FacturaID del detalle existan en Facturas
facturas_faltantes = detalle_facturas.loc[~detalle_facturas['FacturaID'].isin(facturas['FacturaID'])]
if not facturas_faltantes.empty:
    print("⚠️ FacturaID en DetalleFacturas que no existen en Facturas:")
    print(facturas_faltantes['FacturaID'].unique())
else:
    print("✅ Todos los FacturaID del detalle existen en Facturas")

# 2. Verificar que todos los ProductoID del detalle existan en Productos
productos_faltantes = detalle_facturas.loc[~detalle_facturas['ProductoID'].isin(dim_producto['ProductoID'])]
if not productos_faltantes.empty:
    print("⚠️ ProductoID en DetalleFacturas que no existen en Productos:")
    print(productos_faltantes['ProductoID'].unique())
else:
    print("✅ Todos los ProductoID del detalle existen en Productos")

# 3. Verificar que todos los ClienteID en Facturas existan en Clientes
clientes_faltantes = facturas.loc[~facturas['ClienteID'].isin(dim_cliente['ClienteID'])]
if not clientes_faltantes.empty:
    print("⚠️ ClienteID en Facturas que no existen en Clientes:")
    print(clientes_faltantes['ClienteID'].unique())
else:
    print("✅ Todos los ClienteID de Facturas existen en Clientes")

# 4. Verificar que todos los VendedorID en Facturas existan en Vendedores
vendedores_faltantes = facturas.loc[~facturas['VendedorID'].isin(dim_vendedor['VendedorID'])]
if not vendedores_faltantes.empty:
    print("⚠️ VendedorID en Facturas que no existen en Vendedores:")
    print(vendedores_faltantes['VendedorID'].unique())
else:
    print("✅ Todos los VendedorID de Facturas existen en Vendedores")

# 5. Verificar que todos los SucursalID en Facturas existan en Sucursales
sucursales_faltantes = facturas.loc[~facturas['SucursalID'].isin(dim_sucursal['SucursalID'])]
if not sucursales_faltantes.empty:
    print("⚠️ SucursalID en Facturas que no existen en Sucursales:")
    print(sucursales_faltantes['SucursalID'].unique())
else:
    print("✅ Todos los SucursalID de Facturas existen en Sucursales")

# 6. Verificar que todos los MetodoPagoID en Facturas existan en MetodoPago
metodo_faltante = facturas.loc[~facturas['MetodoPagoID'].isin(dim_metodo['MetodoPagoID'])]
if not metodo_faltante.empty:
    print("⚠️ MetodoPagoID en Facturas que no existen en MetodoPago:")
    print(metodo_faltante['MetodoPagoID'].unique())
else:
    print("✅ Todos los MetodoPagoID de Facturas existen en MetodoPago")

print("\n✅ Validación completada.")



🔍 Validando integridad referencial...

✅ Todos los FacturaID del detalle existen en Facturas
✅ Todos los ProductoID del detalle existen en Productos
✅ Todos los ClienteID de Facturas existen en Clientes
✅ Todos los VendedorID de Facturas existen en Vendedores
✅ Todos los SucursalID de Facturas existen en Sucursales
✅ Todos los MetodoPagoID de Facturas existen en MetodoPago

✅ Validación completada.


In [32]:
# --- TOTAL DE VENTAS POR MARCA ---

ventas_por_marca = (
    detalle_facturas
    .merge(dim_producto[['ProductoID', 'MarcaProducto']], on='ProductoID', how='left')
    .groupby('MarcaProducto', as_index=False)['Subtotal']
    .sum()
    .sort_values(by='Subtotal', ascending=False)
)

print("💰 Total de ventas por marca:")
print(ventas_por_marca.head(10))



💰 Total de ventas por marca:
  MarcaProducto      Subtotal
5        Lenovo  117915200000
4            HP   96843000000
3          Dell   86758400000
1         Apple   78021700000
2          Asus   33192000000
0          Acer   22011000000
9       Samsung   12760800000
6           MSI    8525200000
7     Microsoft    8223400000
8         Razer    3936400000


In [33]:
# --- TOP 10 PRODUCTOS MÁS VENDIDOS (por cantidad) ---

top_productos = (
    detalle_facturas
    .merge(dim_producto[['ProductoID', 'NombreProducto']], on='ProductoID', how='left')
    .groupby('NombreProducto', as_index=False)['Cantidad']
    .sum()
    .sort_values(by='Cantidad', ascending=False)
    .head(10)
)

print("🏆 Top 10 productos más vendidos:")
print(top_productos)


🏆 Top 10 productos más vendidos:
               NombreProducto  Cantidad
19            HP Spectre x360      7730
21        Lenovo Legion 5 Pro      5864
22  Lenovo ThinkPad X1 Carbon      5814
23             Lenovo Yoga 7i      5054
17                 HP Omen 16      5015
20           Lenovo IdeaPad 5      4972
15                Dell XPS 13      4530
14         Dell Latitude 7420      4509
18             HP Pavilion 15      4030
12         Dell Alienware m15      3658
